In [1]:
import numpy as np 
import pandas as pd 
import os

In [2]:
!pip install -q transformers > /dev/null
!pip install -q clean-text > /dev/null
!pip install torch-summary > /dev/null

In [3]:
import pandas as pd, numpy as np
import torch
import torch.nn as nn
from transformers import AutoConfig, AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from torch.optim import Adam
from cleantext import clean
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import accuracy_score

**Load the training dataset in a pandas dataframe**

In [4]:
train_path = '../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip'
train = pd.read_csv(train_path)

**Analyse the dataset**

In [18]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
train.isnull().any()

id               False
comment_text     False
toxic            False
severe_toxic     False
obscene          False
threat           False
insult           False
identity_hate    False
dtype: bool

**Define the output classes**

In [7]:
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[classes].values

**Perform dataset cleaning such as removing stop words, emojis, punctuations etc. using clean-text library**

In [8]:
def clean_string(s): return clean(s, no_line_breaks=True, no_urls=True, no_punct=True)

**Custom Dataset loader**

In [9]:
class MyData(torch.utils.data.Dataset):
    def __init__(self, data, label_cols):
        self.data = data
        self.label_cols = label_cols

    def __getitem__(self, item):
        comment = clean_string(self.data.comment_text[item])
        toxic = self.data.toxic[item]
        severe_toxic = self.data.severe_toxic[item]
        obscene = self.data.obscene[item]
        threat = self.data.threat[item]
        insult = self.data.insult[item]
        identity_hate = self.data.identity_hate[item]
        return comment, torch.Tensor([toxic, severe_toxic, obscene, threat, insult, identity_hate])
    
    def __len__(self):
        return len(self.data)

In [10]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
COMMENT = 'comment_text'
label_cols.append(COMMENT)

**Set device for PyTorch Training**

In [11]:
gpu = 0
device = torch.device(gpu if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(gpu)
print(device)

cpu


**Model parameters and hyper-parameters**

In [12]:
MAX_SEQ_LEN = 128
BATCH_SIZE = 64*2
WARMUP_STEPS = 4
EPOCHS = 2
LEARNING_RATE = 5e-3
model_name = 'bert-base-uncased' #'google/electra-small-discriminator'#'bert-base-uncased'
lstm_units = 50

**Define the tokenizer.**
This is dependent on the transformer model passed as input in model_name. Also specify the max sentence length after tokenizing. We are using BERT Word-piece tokenizer for LSTM baseline to avoid any performance disadvantage due to the tokenizer.


In [13]:
import transformers
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=MAX_SEQ_LEN, do_lower_case=True, add_special_tokens=True,
                                                max_length=MAX_SEQ_LEN, pad_to_max_length=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

**Load the training data into torch Dataset and perform train-validation set split.**

In [14]:
train_set = MyData(train, label_cols)
train_set, val_set = torch.utils.data.random_split(train_set, [int(0.9*len(train_set)), len(train_set)-int(0.9*len(train_set))] )
train_set = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_set = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

**Define custom nn model**


In [15]:
class AutoNet(nn.Module):
  def __init__(self, seqLength=MAX_SEQ_LEN, numClasses=6, model_name=model_name, lstm_units=50):
    super(AutoNet, self).__init__()
    if model_name=='google/electra-small-discriminator':
        self.lstm = nn.LSTM(256, lstm_units, dropout=0.1,
                        num_layers=1, bidirectional=True, batch_first=True, bias=False)
    else:
        self.lstm = nn.LSTM(768, lstm_units, dropout=0.1,
                        num_layers=1, bidirectional=True, batch_first=True)
    self.model = nn.Embedding(len(tokenizer), 768)
    self.l1 = nn.Linear(lstm_units * 2, 50)
    self.d1 = nn.Dropout(0.2)
    self.l2 = nn.Linear(50, numClasses)
    self.sigmoid = nn.Sigmoid()

  def forward(self,input_ids, attention_mask=None):
    x = self.model(input_ids)
    x, (hidden, cell) = self.lstm(x)
    x,_ = torch.max(x, dim=1)
    x = self.l1(x)
    x = self.d1(x)
    x = self.l2(x)
    x = self.sigmoid(x)
    return x

**Initialize the model, optimizer and the loss function used.**

In [16]:
model = AutoNet()
model = model.to(device)
model.train()
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
loss_criteria = nn.BCELoss()
loss_criteria = loss_criteria.to(device)

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [17]:
from torchsummary import summary
print(summary(model))

Layer (type:depth-idx)                   Param #
├─LSTM: 1-1                              328,000
├─Embedding: 1-2                         23,440,896
├─Linear: 1-3                            5,050
├─Dropout: 1-4                           --
├─Linear: 1-5                            306
├─Sigmoid: 1-6                           --
Total params: 23,774,252
Trainable params: 23,774,252
Non-trainable params: 0
Layer (type:depth-idx)                   Param #
├─LSTM: 1-1                              328,000
├─Embedding: 1-2                         23,440,896
├─Linear: 1-3                            5,050
├─Dropout: 1-4                           --
├─Linear: 1-5                            306
├─Sigmoid: 1-6                           --
Total params: 23,774,252
Trainable params: 23,774,252
Non-trainable params: 0


**Training the Model**

In [ ]:
model.train()
from tqdm.notebook import tqdm
for epoch in tqdm(range(EPOCHS)):
  count = 0
  total_loss =0 
  model.train()
  for i,data in enumerate(train_set):
    optimizer.zero_grad()
    enc = tokenizer.batch_encode_plus(list(data[0]), pad_to_max_length=True, max_length=MAX_SEQ_LEN, 
                                return_tensors='pt', add_special_tokens=True, return_attention_mask=True,
                                return_token_type_ids=False, )
    input_ids = enc['input_ids'].to(device)
    labels = torch.tensor(data[1]).to(device)
    out = model(input_ids=input_ids) 
    loss = loss_criteria(out, labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.detach().data
    if (i+1)%256==0:
        print(f"Epoch: {epoch}, batch: {i+1}, loss: {total_loss/(BATCH_SIZE)}")
        total_loss = 0

  model.eval()
  all_pred = []
  all_gold = []
  with torch.no_grad():
      for i,data in enumerate(val_set):
        enc = tokenizer.batch_encode_plus(list(data[0]), pad_to_max_length=True, max_length=MAX_SEQ_LEN, return_tensors='pt')
        input_ids = enc['input_ids'].to(device)
        labels = torch.tensor(data[1]).to(device)
        out = model(input_ids=input_ids)
        all_pred.extend(1*(out>0.98).clone().detach().cpu().numpy())
        all_gold.extend((labels.type(torch.LongTensor).detach().cpu().numpy()))

  count=0
  for i in range(len(all_gold)):
    if (all_gold[i]==all_pred[i]).all():
      count+=1
  print("Validation accuracy:", count/len(all_gold))

**Code cell for demo and some error analysis**

In [ ]:
sample_text = "Please get out of here"
clean_txt = clean_string(sample_text)
out1 = tokenizer.encode_plus(list(clean_txt), pad_to_max_length=True, max_length=MAX_SEQ_LEN, return_tensors='pt')
input_ids = out1['input_ids'].to(device)
attention_mask = out1['attention_mask'].to(device)
model.eval()
with torch.no_grad():
    preds = model(input_ids=input_ids, attention_mask=attention_mask)
print(preds)

**Load Test data and analyse**

In [19]:
test_labels_path = '../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip'
test_labels = pd.read_csv(test_labels_path)
test_labels = test_labels.replace(to_replace=-1,value=0)
test_labels.sample(20)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
131184,db442b899dd959ff,0,0,0,0,0,0
53853,5979e856fe936133,0,0,0,0,0,0
37605,3e66a93d9a8b9b51,0,0,0,0,0,0
30069,320d3057c703c820,0,0,0,0,0,0
111330,b9befe5023c77e92,0,0,0,0,0,0
115131,c0256d5fdd84a4b2,0,0,0,0,0,0
125770,d21a063a1b2bc2b5,0,0,0,0,0,0
134140,e03543ab6635c2e5,0,0,0,0,0,0
74456,7c131bc397e4eae4,0,0,0,0,0,0
75632,7e09cfb617e4648f,0,0,0,0,0,0


In [20]:
test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')

test_set = test.merge(test_labels, left_index=True, right_index=True)
test_set = test_set[["id_x", "comment_text", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]
test_set = test_set.reset_index(drop=True)
test_set = test_set.rename(columns={"id_x": "id"})
test_set.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,0,0,0,0,0,0
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,0,0,0,0,0,0
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",0,0,0,0,0,0
3,00017563c3f7919a,":If you have a look back at the source, the in...",0,0,0,0,0,0
4,00017695ad8997eb,I don't anonymously edit articles at all.,0,0,0,0,0,0


**Perform test data cleaning and load into Dataloader**

In [ ]:
test_set['comment_text'] = test_set['comment_text'].apply(clean_string)
ids = test_set['id']
test_set = MyData(test_set, classes)
test_set = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

**Model evaluation**

In [ ]:
model.eval()
all_pred = []
all_gold = []
with torch.no_grad():
    for i,data in enumerate(test_set):
        enc = tokenizer.batch_encode_plus(list(data[0]), pad_to_max_length=True, max_length=MAX_SEQ_LEN, return_tensors='pt')
        input_ids = enc['input_ids'].to(device)
        attention_mask = enc['attention_mask'].to(device)
        labels = torch.tensor(data[1]).to(device)
        out = model(input_ids=input_ids, attention_mask=attention_mask)
        all_pred.extend(out.clone().detach().cpu().numpy())
        all_gold.extend((labels.type(torch.LongTensor).detach().cpu().numpy()))


In [ ]:
accuracy_score(all_gold, (np.array(all_pred) > 0.98))

**Create CSV file for Kaggle Submission**

In [ ]:
target_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
ids = pd.Series(ids)
y_preds = pd.DataFrame(all_pred, columns=target_columns)
final_submission = pd.concat([ids, y_preds], axis=1)
final_submission.head()
final_submission.to_csv('submission.csv', index=False)